In [ ]:
import os
import shutil
from pathlib import Path
from datetime import datetime

# detectar entorno
entorno = "prod"  # o "staging"

# log dinamico por entorno y fecha
hoy = datetime.now().strftime("%Y-%m-%d")
LOG_DIR = Path(f"D:/trading/logs/{entorno}")
LOG_DIR.mkdir(parents=True, exist_ok=True)
LOG_PATH = LOG_DIR / f"notebook_{hoy}.log"


# === CONFIGURACION LOCAL ===
API_KEY = "0428b2bc647a4194b83c890b13d4c68b"  # reemplazar por tu clave Twelve Data
OUTPUT_DIR = Path("D:/trading/data/historic")
CONFIG_PATH = Path("D:/trading/config/json/symbol_groups.json")

# Limpiar carpeta
if OUTPUT_DIR.exists():
    shutil.rmtree(OUTPUT_DIR)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Carpeta {OUTPUT_DIR} limpiada y recreada.")


In [ ]:
import json
import time
import requests
import pandas as pd
from collections import deque

# === LECTURA DE SIMBOLOS ===
with open(CONFIG_PATH, "r") as f:
    grupos = json.load(f)

# === RATE LIMITER ===
class APIRateLimiter:
    def __init__(self, max_requests=5, window_seconds=60):
        self.max_requests = max_requests
        self.window_seconds = window_seconds
        self.timestamps = deque()

    def wait_if_needed(self):
        now = time.time()
        while self.timestamps and now - self.timestamps[0] > self.window_seconds:
            self.timestamps.popleft()
        if len(self.timestamps) >= self.max_requests:
            sleep_time = self.window_seconds - (now - self.timestamps[0]) + 0.5
            print(f"Esperando {sleep_time:.1f}s por rate limit...")
            time.sleep(sleep_time)
        self.timestamps.append(time.time())

rate_limiter = APIRateLimiter()


In [ ]:
def fetch_data(symbol):
    rate_limiter.wait_if_needed()
    
    url = "https://api.twelvedata.com/time_series"
    params = {
        "symbol": symbol,
        "interval": "1day",
        "outputsize": 5000,
        "apikey": API_KEY,
        "format": "JSON"
    }
    
    response = requests.get(url, params=params)
    data = response.json()
    
    if "values" not in data:
        raise ValueError(f"No data for {symbol}: {data.get('message', data)}")
    
    df = pd.DataFrame(data["values"])
    numeric_cols = ["open", "high", "low", "close", "volume"]
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")
    df["datetime"] = pd.to_datetime(df["datetime"])
    df = df.sort_values("datetime")
    return df

def guardar_parquet(df, symbol):
    path = OUTPUT_DIR / f"{symbol}.parquet"
    df.to_parquet(path, index=False, compression="snappy")


In [ ]:
total_ok = 0
total_error = 0

for grupo, simbolos in grupos.items():
    print(f"\n== Procesando {grupo} ({len(simbolos)} simbolos) ==")
    for i, symbol in enumerate(simbolos):
        try:
            df = fetch_data(symbol)
            guardar_parquet(df, symbol)
            print(f"{symbol}: OK")
            total_ok += 1
        except Exception as e:
            print(f"{symbol}: ERROR - {e}")
            total_error += 1
        if i < len(simbolos) - 1:
            time.sleep(15)

print(f"\nFinalizado: {total_ok} OK, {total_error} con error.")
with open(LOG_PATH, "a") as log:
    log.write(f"[{datetime.now()}] Notebook ejecutado correctamente\n")

